---
title: "15-SAS 对二项分布的概率计算与假设检验"
author: "Simon Zhou"
date: "2025-06-07"
date-modified: "2025-06-07"
format: 
    html:
        code-fold: true
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [1]:
%load_ext saspy.sas_magic

## SAS 与 Binomial

在 SAS 中,与二项分布有关的函数为 `probbnm(p,n,r)`,函数中 `p` 为某事件的发生概率,`n` 为样本含量，为阳性事件的例数,该函数可以计算出发生阳性事件的例数从 `0` 到 `r` 的累计概率。利用该函数可以对服从二项分布的数据进行概率计算和假设检验。

## 阳性事件发生的概率

某种医学技能测试的通过率为0.80。今有10例学生参加测试,试分别计算这10例学生中有6、7、8人获得通过的概率。本例 π=0.8,n=10,计算 r=6、7、8的概率.

In [2]:
%%SAS
/*程序15-1*/
data binomial15_1;
    do r=6 to 8;
        p1=probbnml(0.8,10,r);
        p2=probbnml(0.8,10,r-1);
        p=p1-p2;
        output;
    end;
proc print;
    var r p;
run;

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 30048



观测,r,p
1,6,0.08808
2,7,0.20133
3,8,0.30199


## 总体率的区间估计(正态近似法)

根据数理统计学的中心极限定理可得,当 n 较大、π 不接近 0 也不接近 1 时,二项分布 $B(n,π)$ 近似正态分布 $N[n \pi,nπ(1-π)]$,相应的样本率 p 的分布也近似正态分布 $N(\pi,\sigma_{p}^2)$。为此,当 n 较大、p 和 1-p 均不太小,如 np 和 n(1-p) 均>5时,可利用样本率 p 的分布近似正态分布来估计总体率的 1-α 置信区间。

In [3]:
%%SAS
data binomial15_2;
    n=100;
    p=0.45;
    sp=sqrt(p*(1-p)/n);
    u=probit(0.975);
    usp=u*sp;
    lclm=p-usp;
    uclm=p+usp;
proc print;
    var n p sp lclm uclm;
run;

观测,n,p,sp,lclm,uclm
1,100,0.45,0.049749,0.35249,0.54751


### 程序说明

数据集中 n 为观察的患者人数 p 为样本率,驴为率的标准误,u 为置信水准为 0.05 时标准正态分布的双侧界值, lclm 为 95% 置信区间的下限,uclm 为 95% 置信区间的上限。